In [22]:
import pandas as pd

In [23]:
df = pd.read_excel("C:\\Users\\sonihi\\Desktop\\opp_dec.xlsx")
df.dropna(axis=1, how='all', inplace=True)
# df.drop(columns=['Amount', 'Unnamed: 10'], inplace=True)
df.rename(columns={'Unnamed: 9' : 'Note'}, inplace=True)

In [24]:
df['Lead Unit'] = df["Lead Unit"].str.split("/").str[0]

In [25]:
df['Due date'] = pd.to_datetime(df['Due date'], errors='coerce')

C:\Users\sonihi\AppData\Local\Temp\ipykernel_32616\1590513900.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Due date'] = pd.to_datetime(df['Due date'], errors='coerce')


In [26]:
df['Country(ies)'].fillna('Global', inplace=True)

C:\Users\sonihi\AppData\Local\Temp\ipykernel_32616\3537990505.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Country(ies)'].fillna('Global', inplace=True)


In [27]:
df['Lead Unit'].fillna('', inplace=True)

C:\Users\sonihi\AppData\Local\Temp\ipykernel_32616\1563001210.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Lead Unit'].fillna('', inplace=True)


In [28]:
import os
import sys

sys.path.append(os.path.abspath('C:/Projects/Opportunity Tracker/opportunity_tracker'))
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'opportunity_tracker.settings')
import django
django.setup()


In [29]:
from tracker.models import Opportunity, FundingAgency, Country, Unit
from django.contrib.auth.models import User

In [30]:
df.head()

,Funding agency,Title,Country(ies),EoI /RFP,Due date,Lead Unit,Unnamed: 8,Note
0,WHO,Systematic reviews of cost effectiveness studi...,Global,RFP,NaT,,Need to demonstrate 5 yrs experience with GRAD...,NaN
1,UNITAID,Accelerate demand and adoption of new and unde...,Global,RFP,NaT,,Improving access to lifesaving tools for preve...,NaN
2,Enabel,Development of the Strategy and the Digitaliza...,Mauritania,RFP,2024-12-06,,"Team: Damien, Daniella, Simeon, Patrick D. Iss...",NaN
3,WHO,Nigeria Pathway to Malaria Pre-Elimination and...,Nigeria,RFP,NaT,,Possibly for/with Prof. Wellington at U of Lag...,NaN
4,WHO,RFP 2024.16 Mid-Term Review of Unitaid’s 2023-...,Geneva,RFP,2024-12-19,,19/11: CoI with on-going work with Unitaid? Tr...,NaN


In [31]:
from asgiref.sync import sync_to_async
from django.db.models import Q
import numpy as np

In [32]:
@sync_to_async
def get_funding_agency(code):
    try:
        return FundingAgency.objects.filter(
                                                Q(code__iexact=code) | Q(name__iexact=code)
                                            ).first()
    except(FundingAgency.DoesNotExist):
        print(f"Funding Agency {code} does not exist") 

In [33]:
@sync_to_async
def get_countries(country_codes):
    codes = country_codes.split(',')
    countries = []
    try:
        for code in codes:
            country = Country.objects.filter(
            Q(code__iexact=code.strip()) | Q(name__iexact=code.strip())
            ).first()
            if country:
                countries.append(country)
            else:
                print(f"{code} not found")

    except Exception as e:
        print(f"get_countries: {e}")
    finally:
        return countries

In [34]:
@sync_to_async
def get_lead_unit(code):
    try:
        return Unit.objects.filter(
            Q(code__iexact=code) | Q(name__iexact=code)
        ).first()
    except Exception as e:
        print(f"get_lead_unit: {e}")

In [35]:
@sync_to_async
def add_opportunity(ref_no, title, funding_agency, opp_type, due_date, lead_unit, note, countries):
    user = User.objects.get(id=1)
    opportunity = Opportunity.objects.create(
            ref_no=ref_no,
            title=title,
            funding_agency=funding_agency,
            opp_type=opp_type,
            due_date=due_date,
            lead_unit=lead_unit,
            notes=note,
            created_by= user
        )

    opportunity.save()
    opportunity.countries.set(countries)    

In [36]:
for ind, opp in df.iterrows():
    funding_agency = await get_funding_agency(opp['Funding agency'])
    title = opp['Title']
    countries = await get_countries(opp['Country(ies)'])
    opp_type = opp['EoI /RFP']
    due_date = opp['Due date'] if opp['Due date'] is not pd.NaT else None
    lead_unit = await get_lead_unit(opp['Lead Unit']) if opp['Lead Unit'] != np.nan else None
    note = opp['Note']
    ref_no = f"{funding_agency}/2024/11/{ind}"
    
    try:
        await add_opportunity(ref_no, title, funding_agency, opp_type, due_date, lead_unit, note, countries)
    except Exception as e:
        print(f"Error {ind}: {e}")

An unhandled exception occurred: [Errno 11001] getaddrinfo failed
An unhandled exception occurred: [Errno 11001] getaddrinfo failed
An unhandled exception occurred: [Errno 11001] getaddrinfo failed


c:\Projects\Opportunity Tracker\venv\Lib\site-packages\django\db\models\fields\__init__.py:1665: RuntimeWarning: DateTimeField Opportunity.due_date received a naive datetime (2024-12-06 00:00:00) while time zone support is active.
  warnings.warn(
c:\Projects\Opportunity Tracker\venv\Lib\site-packages\django\db\models\fields\__init__.py:1665: RuntimeWarning: DateTimeField Opportunity.due_date received a naive datetime (2024-12-19 00:00:00) while time zone support is active.
  warnings.warn(
c:\Projects\Opportunity Tracker\venv\Lib\site-packages\django\db\models\fields\__init__.py:1665: RuntimeWarning: DateTimeField Opportunity.due_date received a naive datetime (2024-12-20 00:00:00) while time zone support is active.
  warnings.warn(
c:\Projects\Opportunity Tracker\venv\Lib\site-packages\django\db\models\fields\__init__.py:1665: RuntimeWarning: DateTimeField Opportunity.due_date received a naive datetime (2024-12-15 00:00:00) while time zone support is active.
  warnings.warn(
c:\Proje

An unhandled exception occurred: [Errno 11001] getaddrinfo failed
Geneva  not found
An unhandled exception occurred: [Errno 11001] getaddrinfo failed
An unhandled exception occurred: [Errno 11001] getaddrinfo failed
An unhandled exception occurred: [Errno 11001] getaddrinfo failed
An unhandled exception occurred: [Errno 11001] getaddrinfo failed
An unhandled exception occurred: [Errno 11001] getaddrinfo failed
 CdI not found
An unhandled exception occurred: [Errno 11001] getaddrinfo failed
An unhandled exception occurred: [Errno 11001] getaddrinfo failed
An unhandled exception occurred: [Errno 11001] getaddrinfo failed
An unhandled exception occurred: [Errno 11001] getaddrinfo failed


c:\Projects\Opportunity Tracker\venv\Lib\site-packages\django\db\models\fields\__init__.py:1665: RuntimeWarning: DateTimeField Opportunity.due_date received a naive datetime (2025-03-01 00:00:00) while time zone support is active.
  warnings.warn(
c:\Projects\Opportunity Tracker\venv\Lib\site-packages\django\db\models\fields\__init__.py:1665: RuntimeWarning: DateTimeField Opportunity.due_date received a naive datetime (2024-11-21 00:00:00) while time zone support is active.
  warnings.warn(
c:\Projects\Opportunity Tracker\venv\Lib\site-packages\django\db\models\fields\__init__.py:1665: RuntimeWarning: DateTimeField Opportunity.due_date received a naive datetime (2024-11-15 00:00:00) while time zone support is active.
  warnings.warn(
c:\Projects\Opportunity Tracker\venv\Lib\site-packages\django\db\models\fields\__init__.py:1665: RuntimeWarning: DateTimeField Opportunity.due_date received a naive datetime (2024-11-07 00:00:00) while time zone support is active.
  warnings.warn(
c:\Proje

An unhandled exception occurred: [Errno 11001] getaddrinfo failed
An unhandled exception occurred: [Errno 11001] getaddrinfo failed
An unhandled exception occurred: [Errno 11001] getaddrinfo failed
Error 16: value too long for type character varying(50)
An unhandled exception occurred: [Errno 11001] getaddrinfo failed
An unhandled exception occurred: [Errno 11001] getaddrinfo failed
An unhandled exception occurred: [Errno 11001] getaddrinfo failed
An unhandled exception occurred: [Errno 11001] getaddrinfo failed
An unhandled exception occurred: [Errno 11001] getaddrinfo failed
An unhandled exception occurred: [Errno 11001] getaddrinfo failed
An unhandled exception occurred: [Errno 11001] getaddrinfo failed
An unhandled exception occurred: [Errno 11001] getaddrinfo failed
An unhandled exception occurred: [Errno 11001] getaddrinfo failed
East and Southern Africa not found


c:\Projects\Opportunity Tracker\venv\Lib\site-packages\django\db\models\fields\__init__.py:1665: RuntimeWarning: DateTimeField Opportunity.due_date received a naive datetime (2024-08-26 00:00:00) while time zone support is active.
  warnings.warn(
c:\Projects\Opportunity Tracker\venv\Lib\site-packages\django\db\models\fields\__init__.py:1665: RuntimeWarning: DateTimeField Opportunity.due_date received a naive datetime (2024-07-11 00:00:00) while time zone support is active.
  warnings.warn(
c:\Projects\Opportunity Tracker\venv\Lib\site-packages\django\db\models\fields\__init__.py:1665: RuntimeWarning: DateTimeField Opportunity.due_date received a naive datetime (2024-07-10 00:00:00) while time zone support is active.
  warnings.warn(
c:\Projects\Opportunity Tracker\venv\Lib\site-packages\django\db\models\fields\__init__.py:1665: RuntimeWarning: DateTimeField Opportunity.due_date received a naive datetime (2024-07-08 00:00:00) while time zone support is active.
  warnings.warn(
c:\Proje

An unhandled exception occurred: [Errno 11001] getaddrinfo failed
Remote not found
An unhandled exception occurred: [Errno 11001] getaddrinfo failed
Remote not found
An unhandled exception occurred: [Errno 11001] getaddrinfo failed
Remote not found
An unhandled exception occurred: [Errno 11001] getaddrinfo failed
An unhandled exception occurred: [Errno 11001] getaddrinfo failed
An unhandled exception occurred: [Errno 11001] getaddrinfo failed
An unhandled exception occurred: [Errno 11001] getaddrinfo failed
Remote not found
An unhandled exception occurred: [Errno 11001] getaddrinfo failed
An unhandled exception occurred: [Errno 11001] getaddrinfo failed
An unhandled exception occurred: [Errno 11001] getaddrinfo failed
selected countries not found
An unhandled exception occurred: [Errno 11001] getaddrinfo failed
An unhandled exception occurred: [Errno 11001] getaddrinfo failed
An unhandled exception occurred: [Errno 11001] getaddrinfo failed
Cameroun not found
An unhandled exception occ

c:\Projects\Opportunity Tracker\venv\Lib\site-packages\django\db\models\fields\__init__.py:1665: RuntimeWarning: DateTimeField Opportunity.due_date received a naive datetime (2024-11-25 00:00:00) while time zone support is active.
  warnings.warn(
c:\Projects\Opportunity Tracker\venv\Lib\site-packages\django\db\models\fields\__init__.py:1665: RuntimeWarning: DateTimeField Opportunity.due_date received a naive datetime (2024-11-30 00:00:00) while time zone support is active.
  warnings.warn(
c:\Projects\Opportunity Tracker\venv\Lib\site-packages\django\db\models\fields\__init__.py:1665: RuntimeWarning: DateTimeField Opportunity.due_date received a naive datetime (2024-12-05 00:00:00) while time zone support is active.
  warnings.warn(
c:\Projects\Opportunity Tracker\venv\Lib\site-packages\django\db\models\fields\__init__.py:1665: RuntimeWarning: DateTimeField Opportunity.due_date received a naive datetime (2024-11-26 00:00:00) while time zone support is active.
  warnings.warn(


An unhandled exception occurred: [Errno 11001] getaddrinfo failed
An unhandled exception occurred: [Errno 11001] getaddrinfo failed
An unhandled exception occurred: [Errno 11001] getaddrinfo failed
An unhandled exception occurred: [Errno 11001] getaddrinfo failed
CdI - two regions Loh Dijouba and Haut Sassandra not found
An unhandled exception occurred: [Errno 11001] getaddrinfo failed
An unhandled exception occurred: [Errno 11001] getaddrinfo failed


c:\Projects\Opportunity Tracker\venv\Lib\site-packages\django\db\models\fields\__init__.py:1665: RuntimeWarning: DateTimeField Opportunity.due_date received a naive datetime (2022-11-01 00:00:00) while time zone support is active.
  warnings.warn(
